In [1]:
sc

In [2]:
import geopandas as gpd
import pyproj
import shapely.geometry as geom
import csv
import datetime

In [24]:
collisions = 'data/collisions.csv'
jan = 'data/1january.csv'
feb = 'data/2february.csv'
march = 'data/3march.csv'
april = 'data/4april.csv'
may = 'data/5may.csv'
june = 'data/6june.csv'

collisions = sc.textFile(collisions, use_unicode=False).cache()
janTrips = sc.textFile(jan,use_unicode=False).cache()
febTrips = sc.textFile(feb,use_unicode=False).cache()
marchTrips = sc.textFile(march,use_unicode=False).cache()
aprilTrips = sc.textFile(april,use_unicode=False).cache()
mayTrips = sc.textFile(may,use_unicode=False).cache()
juneTrips = sc.textFile(june,use_unicode=False).cache()

In [69]:
def extractTrips(partitionId,partition):
    if partitionId == 0:
        partition.next()
    import csv
    reader = csv.reader(partition)
    for row in reader:
        if row[1] and row[2] !='0' and row[6] and row [7] != '0':
            pickup = row[1][0:6],row[2][0:7]
            dropoff = row[6][0:6],row[7][0:7]       
            yield (pickup,1)
            yield (dropoff,1)

In [35]:
# def extractTrips(partitionId,partition):
#     if partitionId == 0:
#         partition.next()
#     import csv
#     reader = csv.reader(partition)
#     for row in reader:
#         if row[1] and row[2] !='0' and row[6] and row [7] != '0':
#             pickup = row[1],row[2]
#             dropoff = row[6],row[7]
#             yield (pickup,1)
#             yield (dropoff,1)


In [61]:
def extractCyclists(partitionId,partition):
    if partitionId == 0:
        partition.next()
    import csv
    reader = csv.reader(partition)
    for row in reader:
        if row[14]!= '' and row[14] != '0':
            lat = row[6].split(',')[0][1:7] + '' , '' + row[6].split(',')[1][0:7]
            injured = int(row[14]) + int(row[15])    # includes injury and death
            yield (lat),injured

In [63]:
cyclistAccidents = collisions.mapPartitionsWithIndex(extractCyclists).reduceByKey(lambda x,y: x+y)


In [64]:
cyclistAccidents.take(20)

[(('40.871', '-73.858'), 2),
 (('40.749', '-73.864'), 6),
 (('40.760', '-73.969'), 6),
 (('40.754', '-73.976'), 1),
 (('40.684', '-73.838'), 2),
 (('40.721', '-73.956'), 2),
 (('40.771', '-73.908'), 4),
 (('40.655', '-73.766'), 1),
 (('40.635', '-74.006'), 2),
 (('40.735', '-73.869'), 1),
 (('40.757', '-73.750'), 1),
 (('40.632', '-73.888'), 1),
 (('40.856', '-73.906'), 1),
 (('40.880', '-73.883'), 1),
 (('40.809', '-73.947'), 1),
 (('40.770', '-73.927'), 1),
 (('40.610', '-74.146'), 1),
 (('40.672', '-73.903'), 2),
 (('40.728', '-74.009'), 1),
 (('40.748', '-73.895'), 1)]

In [15]:
janTrips.count()
#cyclistAccidents.join(janTrips).take(20)


10482803

In [31]:
january_taxi = janTrips.mapPartitionsWithIndex(extractTrips).reduceByKey(lambda x, y: x+y)
january_taxi.take(10)

[(('40.8113', '-73.9311'), 3),
 (('40.6877', '-73.9438'), 12),
 (('40.7616', '-73.9650'), 60),
 (('40.6947', '-73.9192'), 3),
 (('40.7456', '-73.9827'), 29),
 (('40.7612', '-73.9598'), 30),
 (('40.7286', '-73.9763'), 58),
 (('40.7458', '-73.9790'), 12),
 (('40.7043', '-73.9872'), 1),
 (('40.7093', '-74.0601'), 1)]

In [32]:
january_taxi.count()

836723

In [50]:
january_taxi_old = janTrips.mapPartitionsWithIndex(extractTrips2).reduceByKey(lambda x, y: x+y)
january_taxi_old.count()

In [70]:
jan_taxi = janTrips.mapPartitionsWithIndex(extractTrips).reduceByKey(lambda x, y: x+y)
#jan_taxi.count()

In [46]:
test_taxi.take(20)
test_taxi.values().sum()

21813716

In [47]:
january_taxi.values().sum()

21813716

In [52]:
jan_taxi.take(20)

[(('40.8113', '-73.9311'), 3),
 (('40.6877', '-73.9438'), 12),
 (('40.7616', '-73.9650'), 60),
 (('40.6947', '-73.9192'), 3),
 (('40.7456', '-73.9827'), 29),
 (('40.7612', '-73.9598'), 30),
 (('40.7286', '-73.9763'), 58),
 (('40.7458', '-73.9790'), 12),
 (('40.7043', '-73.9872'), 1),
 (('40.7093', '-74.0601'), 1),
 (('40.7593', '-73.9410'), 2),
 (('40.7364', '-73.9842'), 8),
 (('40.6591', '-73.9148'), 2),
 (('40.7305', '-73.9988'), 23),
 (('40.7884', '-73.9512'), 4),
 (('40.7269', '-73.8290'), 1),
 (('40.8450', '-73.9407'), 9),
 (('40.7767', '-73.9482'), 4),
 (('40.7286', '-73.9959'), 1),
 (('40.7780', '-73.9136'), 1)]

In [71]:
feb_taxi = febTrips.mapPartitionsWithIndex(extractTrips).reduceByKey(lambda x, y: x+y)
#feb_taxi.count()

In [55]:
feb_taxi.values().sum()

22764098

In [72]:
jan_feb = feb_taxi.join(jan_taxi).mapValues(lambda x: x[0]+x[1])

In [57]:
jan_feb.count()

591319

In [58]:
jan_feb.values().sum()

43908352

In [59]:
jan_feb.take(20)

[(('40.7719', '-73.9030'), 3),
 (('40.6827', '-73.9417'), 7),
 (('40.7068', '-74.0095'), 31),
 (('40.7492', '-73.9724'), 284),
 (('40.7173', '-73.9942'), 52),
 (('40.8085', '-73.9262'), 2),
 (('40.7541', '-73.9653'), 7),
 (('40.7710', '-73.9899'), 4),
 (('40.7221', '-73.9500'), 63),
 (('40.7269', '-73.8529'), 3),
 (('40.7429', '-73.9803'), 184),
 (('40.7197', '-74.008'), 8),
 (('40.6167', '-74.0232'), 3),
 (('40.6582', '-73.9208'), 2),
 (('40.7444', '-73.9932'), 6),
 (('40.7161', '-74.0028'), 11),
 (('40.6692', '-73.9352'), 4),
 (('40.7179', '-74.0036'), 2),
 (('40.8126', '-73.9510'), 3),
 (('40.7840', '-73.9705'), 180)]

In [73]:
total_data = jan_feb.join(cyclistAccidents)

In [74]:
total_data.count()

7652